In [2]:
title = []
skills = []
company = []

In [55]:
def r_time():
    return random.randint(1,8)

In [4]:
def next_page():
    for i in range(10):
        next_click = driver.find_element_by_link_text('Next')
        action = ActionChains(driver)
        action.click(on_element=next_click)
        action.perform()
        time.sleep(r_time())

In [5]:
def get_items(html_source):
    soup = BeautifulSoup(html_source, 'html.parser')
    jobt = soup.select('div[data-automation=detailsTitle]')
    for i in jobt:
        title.append(i.find('h1').get_text())
        company.append(i.find('span').get_text())
    desc = soup.select('div[data-automation=jobDescription]')
    for i in desc:
        reqs_i = []
        lists = i.findAll('ul')
        for j in lists:
            items = j.findAll('li')
            for k in items:
                reqs_i.append(k.get_text())
        skills.append(reqs_i)

In [10]:
def scrape(url):
    driver.get(url)
    for i in range(0,30):
        try:
            css = 'article[data-automation=job-card-' + str(i) +']'
            time.sleep(r_time())
            element = driver.find_element_by_css_selector(css)
#            k = i + 1
#            xpath = '/html/body/div[1]/div/div/div[2]/div[2]/div/div/div[1]/div/div/div/div[2]/div[3]/div/div['+ str(k) +']/div/div/article/div/div/div[1]/div[1]'
            elems = element.find_element_by_tag_name('div')
            action = ActionChains(driver)
            action.click(on_element=elems)
            action.perform()
            time.sleep(r_time())
            html_source = driver.page_source
            get_items(html_source)
            time.sleep(r_time())
        except:
            continue

In [96]:
PATH = "C:/Users/hbalcera/Desktop/S4/chromedriver"
driver = webdriver.Chrome(PATH)

base = 'https://www.jobstreet.com.ph/en/job-search/data-scientist-jobs/4/'

scrape(base)
driver.quit()

In [97]:
Titledf = pd.DataFrame({'title':title})
Skillsdf = pd.DataFrame({'Skills':skills})
Companydf = pd.DataFrame({'company':company})

In [98]:
jobstreet2 = pd.concat([Skillsdf, Titledf, Companydf], axis=1)

In [99]:
js_df = pd.concat([js_df, jobstreet2], axis=0)

In [100]:
len(js_df)

2215

In [101]:
cleaning = pd.DataFrame(columns=['lower','num','punct'])

In [102]:
cleaning['liststring'] = [','.join(map(str, l)) for l in js_df['Skills']]

cleaning['lower'] = [i.lower() for i in cleaning['liststring']]
cleaning['num'] = [re.sub(r'\w*\d\w*', '', i).strip() for i in cleaning['lower']]

import string
js_df['Skills'] = [s.translate(str.maketrans('','',string.punctuation)) for s in cleaning['num']]

In [103]:
js_df.head()

,Skills,title,company
0,scope and analyze business problems by creatin...,Data Scientist,"Smart Communications, Inc."
1,scope and analyze business problems by creatin...,Data Scientist,"Smart Communications, Inc."
2,develop machine learning models ml models to s...,Senior Data Scientist,"Novare Technologies, Inc."
3,collaborate with crossfunctional stakeholders ...,Data Scientist,EMAPTA
4,lead a team of dav associates and analysts and...,Data Analytics & Visualization Supervisor,Curran Daly & Associates


In [104]:
js_df.to_csv('jobstreet.csv')

In [ ]:
questions['votes'] = pd.to_numeric(questions['votes'])